In [1]:
import sys
sys.path.append('../')
import os
os.environ['REDIS_OM_URL'] = "redis://:QzmCUD3C3RdsR@localhost:6379"
from typing import get_args
from pydantic import ValidationError
from sotopia.database import AgentProfile, EnvironmentProfile, RelationshipProfile, EpisodeLog
from sotopia.database import (
    episodes_to_csv, 
    episodes_to_jsonl, 
    agentprofiles_to_csv, 
    agentprofiles_to_jsonl, 
    environmentprofiles_to_csv, 
    environmentprofiles_to_jsonl, 
    relationshipprofiles_to_csv, 
    relationshipprofiles_to_jsonl,
    envagnetcombostorage_to_csv,
    envagnetcombostorage_to_jsonl,
    jsonl_to_episodes,
    jsonl_to_agentprofiles,
    jsonl_to_environmentprofiles,
    jsonl_to_relationshipprofiles,
    jsonl_to_envagnetcombostorage,
)
from sotopia.database.env_agent_combo_storage import EnvAgentComboStorage
from typing import List, Dict, Any, Literal
from pprint import pprint

### Episodes Serialization

In [2]:
LLM_Name = Literal[
    "togethercomputer/llama-2-7b-chat",
    "togethercomputer/llama-2-70b-chat",
    "togethercomputer/mpt-30b-chat",
    "gpt-3.5-turbo",
    "text-davinci-003",
    "gpt-4",
    "gpt-4-turbo",
    "human",
    "redis",
]
def _is_valid_episode_log_pk(pk: str) -> bool:
    try:
        episode = EpisodeLog.get(pk=pk)
    except ValidationError:
        return False
    try:
        tag = episode.tag
        model_1, model_2, version = tag.split("_", maxsplit=2)
        if (
            model_1 in get_args(LLM_Name)
            and model_2 in get_args(LLM_Name)
            and version == "v0.0.1_clean"
        ):
            return True
        else:
            return False
    except (ValueError, AttributeError):
        # ValueError: tag has less than 3 parts
        # AttributeError: tag is None
        return False


episodes: list[EpisodeLog] = [
    EpisodeLog.get(pk=pk)
    for pk in filter(_is_valid_episode_log_pk, EpisodeLog.all_pks())
]
pprint(episodes[0])

EpisodeLog(pk='01H9FG15A2NDTNH8K6F2T5MZN3', environment='01H7VFHNN7XTR99319DS8KZCQM', agents=['01H5TNE5PAZABGW79HJ07TACCZ', '01H5TNE5P83CZ1TDBVN74NGEEJ'], tag='togethercomputer/mpt-30b-chat_togethercomputer/llama-2-70b-chat_v0.0.1_clean', models=['gpt-4', 'togethercomputer/mpt-30b-chat', 'togethercomputer/llama-2-70b-chat'], messages=[[('Environment', 'Mia Davis', "\nHere is the context of this interaction:\nScenario: Two friends are camping in the wilderness and the temperature drops significantly at night\nParticipants: Mia Davis and William Brown\nMia Davis's background: Mia Davis is a 50-year-old female high school principal. She/her pronouns. Mia Davis has two cats. Personality and values description: Mia Davis, an extraverted stickler for routines, values tradition and authority. Her decision-making style is decisive and direct. Mia's secrets: Part of a rebellious punk rock band in her youth\nWilliam Brown's background: William Brown is a 35-year-old male chef. He/him pronouns. W

In [3]:
episodes_to_csv(episodes, "../data/sotopia_episodes_v1.csv")
episodes_to_jsonl(episodes, "../data/sotopia_episodes_v1.jsonl")
rebuild_episodes = jsonl_to_episodes("../data/sotopia_episodes_v1.jsonl")
pprint(rebuild_episodes[0])

EpisodeLog(pk='01H9FG15A2NDTNH8K6F2T5MZN3', environment='01H7VFHNN7XTR99319DS8KZCQM', agents=['01H5TNE5PAZABGW79HJ07TACCZ', '01H5TNE5P83CZ1TDBVN74NGEEJ'], tag='togethercomputer/mpt-30b-chat_togethercomputer/llama-2-70b-chat_v0.0.1_clean', models=['gpt-4', 'togethercomputer/mpt-30b-chat', 'togethercomputer/llama-2-70b-chat'], messages=[[('Environment', 'Mia Davis', "\nHere is the context of this interaction:\nScenario: Two friends are camping in the wilderness and the temperature drops significantly at night\nParticipants: Mia Davis and William Brown\nMia Davis's background: Mia Davis is a 50-year-old female high school principal. She/her pronouns. Mia Davis has two cats. Personality and values description: Mia Davis, an extraverted stickler for routines, values tradition and authority. Her decision-making style is decisive and direct. Mia's secrets: Part of a rebellious punk rock band in her youth\nWilliam Brown's background: William Brown is a 35-year-old male chef. He/him pronouns. W

## Relationship Profile Serialization

In [4]:
pks = RelationshipProfile.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(RelationshipProfile.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

RelationshipProfile(pk='01H6HH3KG4M9E1J5H25WHQG6NN', agent_1_id='01H5TNE5PT06B3QPXJ65HHACV7', agent_2_id='01H5TNE5P6KZKR2AEY6SZB83H0', relationship=<RelationshipType.romantic_relationship: 4>, background_story="Donovan and Oliver met at a local cafe where they both enjoy spending their leisure time. They have shared a few conversations talking about their interests and work. Donovan appreciates Oliver's meticulousness and analytical thinking, while Oliver admires Donovan's skills in coding. They have not developed a close friendship but are familiar with each other's names and backgrounds.")


In [5]:
relationshipprofiles_to_csv(res, "../data/sotopia_relationshipprofiles_v1.csv")
relationshipprofiles_to_jsonl(res, "../data/sotopia_relationshipprofiles_v1.jsonl")
relationship_profiles = jsonl_to_relationshipprofiles("../data/sotopia_relationshipprofiles_v1.jsonl")
pprint(relationship_profiles[0])

RelationshipProfile(pk='01H6HH3KG4M9E1J5H25WHQG6NN', agent_1_id='01H5TNE5PT06B3QPXJ65HHACV7', agent_2_id='01H5TNE5P6KZKR2AEY6SZB83H0', relationship=<RelationshipType.romantic_relationship: 4>, background_story="Donovan and Oliver met at a local cafe where they both enjoy spending their leisure time. They have shared a few conversations talking about their interests and work. Donovan appreciates Oliver's meticulousness and analytical thinking, while Oliver admires Donovan's skills in coding. They have not developed a close friendship but are familiar with each other's names and backgrounds.")


## Agents Profile Serialization

In [6]:
pks = AgentProfile.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(AgentProfile.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

AgentProfile(pk='01H5TNE5PBXGRD41HXQC1ZXHVN', first_name='Ethan', last_name='Johnson', age=34, occupation='Chef', gender='Man', gender_pronoun='He/him', public_info='Ethan Johnson is famous for cooking Italian food.', big_five='Openness to Experience - High; Conscientiousness - Low; Extraversion - High; Agreeableness - Low; Neuroticism - Low', moral_values=['Fairness/Reciprocity'], schwartz_personal_values=['Power'], personality_and_values='Ethan Johnson, a creative yet somewhat reserved individual, values power and fairness. He likes to analyse situations before deciding.', decision_making_style='Analytical', secret='He is a closeted romance novel author.', model_id='', mbti='ENTP')


In [7]:
agentprofiles_to_csv(res, "../data/sotopia_agentprofiles_v1.csv")
agentprofiles_to_jsonl(res, "../data/sotopia_agentprofiles_v1.jsonl")
agent_profiles = jsonl_to_agentprofiles("../data/sotopia_agentprofiles_v1.jsonl")
pprint(agent_profiles[0])

AgentProfile(pk='01H5TNE5PBXGRD41HXQC1ZXHVN', first_name='Ethan', last_name='Johnson', age=34, occupation='Chef', gender='Man', gender_pronoun='He/him', public_info='Ethan Johnson is famous for cooking Italian food.', big_five='Openness to Experience - High; Conscientiousness - Low; Extraversion - High; Agreeableness - Low; Neuroticism - Low', moral_values=['Fairness/Reciprocity'], schwartz_personal_values=['Power'], personality_and_values='Ethan Johnson, a creative yet somewhat reserved individual, values power and fairness. He likes to analyse situations before deciding.', decision_making_style='Analytical', secret='He is a closeted romance novel author.', model_id='', mbti='ENTP')


## Environment Profile Serialization

In [8]:
pks = EnvironmentProfile.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(EnvironmentProfile.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

EnvironmentProfile(pk='01H7VFHN1PK2FXY7TPWQK343BQ', codename='borrow_money', source='hand-craft', scenario='Conversation between two friends at a tea party', agent_goals=['Borrow money (<extra_info>Extra information: you need $3000 to support life.</extra_info>)', 'Maintain financial stability while maintaining friendship (<extra_info>Extra information: you only have $2000 available right now. <clarification_hint>Hint: you can not lend all $2000 since you still need to maintain your financial stability.</clarification_hint></extra_info>)'], relationship=<RelationshipType.friend: 3>, age_constraint='[(18, 70), (18, 70)]', occupation_constraint='nan', agent_constraint=None)


In [9]:
environmentprofiles_to_csv(res, "../data/sotopia_environmentprofiles_v1.csv")
environmentprofiles_to_jsonl(res, "../data/sotopia_environmentprofiles_v1.jsonl")
environment_profiles = jsonl_to_environmentprofiles("../data/sotopia_environmentprofiles_v1.jsonl")
pprint(environment_profiles[0])

EnvironmentProfile(pk='01H7VFHN1PK2FXY7TPWQK343BQ', codename='borrow_money', source='hand-craft', scenario='Conversation between two friends at a tea party', agent_goals=['Borrow money (<extra_info>Extra information: you need $3000 to support life.</extra_info>)', 'Maintain financial stability while maintaining friendship (<extra_info>Extra information: you only have $2000 available right now. <clarification_hint>Hint: you can not lend all $2000 since you still need to maintain your financial stability.</clarification_hint></extra_info>)'], relationship=<RelationshipType.friend: 3>, age_constraint='[(18, 70), (18, 70)]', occupation_constraint='nan', agent_constraint=None)


## EnvAgentComboStorage

In [10]:
pks = EnvAgentComboStorage.all_pks()
pks = list(pks)
res = []
for pk in pks:
    try:
        res.append(EnvAgentComboStorage.get(pk=pk))
    except:
        print("error")
        pass
pprint(res[0])

EnvAgentComboStorage(pk='01H7ZDKVZJPQT6G109BKAZTHCF', env_id='01H7VFHN94S6Z5T6ZNC23238NT', agent_ids=['01H5TNE5PRCAF1CK5ERS5MVZ22', '01H5TNE5Q1J7Z7Q12WA1W90MR9'])


In [11]:
envagnetcombostorage_to_csv(res, "../data/sotopia_envagnetcombostorage_v1.csv")
envagnetcombostorage_to_jsonl(res, "../data/sotopia_envagnetcombostorage_v1.jsonl")
envagnetcombostorage = jsonl_to_envagnetcombostorage("../data/sotopia_envagnetcombostorage_v1.jsonl")
pprint(envagnetcombostorage[0])

EnvAgentComboStorage(pk='01H7ZDKVZJPQT6G109BKAZTHCF', env_id='01H7VFHN94S6Z5T6ZNC23238NT', agent_ids=['01H5TNE5PRCAF1CK5ERS5MVZ22', '01H5TNE5Q1J7Z7Q12WA1W90MR9'])
